In [76]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir

# 
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

from sklearn import linear_model
from scipy import stats
import numpy as np
import matplotlib.patches as mpatches

import pylab as P


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# visualize results module
from visualize import visualize_svm as Visualize

# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [12]:
##########################################################################
##########################################################################
##########################################################################

def compute_significance(data,
                         significance):

        #print ("self.data: ", data.shape)
        
        mean = data.mean(1)
        #
        sig = []
        for k in range(data.shape[0]):
            #res = stats.ks_2samp(self.data[k],
            #                     control)
            #res = stats.ttest_ind(first, second, axis=0, equal_var=True)

            #
            res = scipy.stats.ttest_1samp(data[k], 0.5)

            sig.append(res[1])


        sig_save = np.array(sig).copy()
        #print ("Self sig save: ", sig_save.shape)

        # multiple hypothesis test
        temp = np.array(sig)
        #print ("data into multi-hypothesis tes:", temp.shape)
        temp2 = multipletests(temp, alpha=significance, method='fdr_bh')
        sig = temp2[1]

        #
        sig=np.array(sig)[None]

        #
        thresh = significance
        idx = np.where(sig>thresh)
        sig[idx] = np.nan

        #
        idx = np.where(mean<0.5)
        sig[:,idx] = np.nan
        #print ("SIG: ", sig.shape)

        # find earliest 
        earliest_continuous = 0
        for k in range(sig.shape[1]-1,0,-1):
            if sig[0][k]<=significance:
                earliest_continuous = k
            else:
                break
                
        earliest_continuous = -(sig.shape[1]-earliest_continuous)/30.
                
        return sig, earliest_continuous
    
    
class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """

    def __init__(self, *args, **kwargs):
        if not "fit_intercept" in kwargs:
            kwargs['fit_intercept'] = False
        super(LinearRegression, self)\
                .__init__(*args, **kwargs)

    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)

        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([
            np.sqrt(np.diagonal(sse[i] * np.linalg.inv(np.dot(X.T, X))))
                                                    for i in range(sse.shape[0])
                    ])

        self.t = self.coef_ / se
        self.p = 2 * (1 - stats.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1]))
        return self
    
 

def plot_edt_distributions():

    fig = plt.figure(figsize=(20,12))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(2,3,ctr+1)
        fnames = []
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_lockout.npz')
        fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time_concatenated.npz')

        # 
        shifts=[+15,+15,0]
        linestyles = ['-','-','-']
        titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []
        for ctr_type, fname in enumerate(fnames):
            #print (" ", animal_id, fname)
            data = np.load(fname, allow_pickle=True)

            # 
            clr = vis.colors[ctr_type]
            res = vis.plot_first_decoding_time_curves(data,
                                                        titles[ctr_type],
                                                        shifts[ctr_type],
                                                        linestyles[ctr_type],
                                                        clr)
            edts.append(res[0])
            yvals.append(res[1])

        if ctr==0:
            plt.legend(fontsize=legend_size)

        temp = np.hstack(yvals)
        ymax = np.max(temp)
        #print ("ymax", ymax)
        #
        plt.xlim(-15,0)
        plt.ylim(bottom=0)    
        plt.xticks([])
        plt.yticks([])
        plt.plot([-3,-3],[0,ymax*1.1],'--',linewidth=5,c='grey')
        plt.plot([-10,-10],[0,ymax*1.1],'--',linewidth=5,c='grey')
        #
        #ax2=
        ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

        patches = []
        res = stats.ks_2samp(edts[0], edts[1])
        print (res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='blue', label=label_))

        res = stats.ks_2samp(edts[0], edts[2])
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(color='red', label=label_))

        #print ('')
        plt.legend(handles=patches,
                   fontsize=legend_size)



        plt.xlim(-15,0)
        plt.ylim(bottom=0)
        plt.xticks([])
        plt.yticks([])

    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first.svg',dpi=1200)
        plt.close()
        
     

In [75]:
################################

fnames = [
#'/home/cat/IA1_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
#'/home/cat/IA2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IA3_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
#'/home/cat/IJ1_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IJ2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
#'/home/cat/AQ2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IA1_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IA2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IA3_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IJ1_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
# '/home/cat/IJ2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz',
'/home/cat/AQ2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz'
]

clrs = ['black','blue','red','green','magenta','brown']
ctr=0
fig=plt.figure()
for fname in fnames:
    print ("fname", fname)
    data = np.load(fname)
    acc = data['accuracy']
    predictions = data['predictions']
    print (predictions.shape)
    print (acc.shape)

    
    # 
    mean = acc.mean(1)
    std = np.std(acc, axis=1)

    ax=plt.subplot(111)
    t = np.arange(acc.shape[0])/30-29
    plt.plot(t,mean,
            color=clrs[ctr]
            #color='blue'
            )
    ax.fill_between(t, mean+std,
                   mean-std,
                   color=clrs[ctr],
                   #color='blue',
                   alpha=.2)
    plt.plot([t[0],t[-1]],[0.5,0.5],'--')

    plt.xlim(-20,0)
    plt.ylim(0.4,1)
    ctr+=1

if False:
    plt.savefig(fname[:13]+'_superres.svg')
    plt.close()

else:
    plt.show()

fname /home/cat/AQ2_[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]_super_res.npz
(900, 13504)
(900, 10)


In [65]:
#d = np.load('/home/cat/IJ1_stacks_all.npy')

data = np.load('/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz_rnn_training.npz',
             allow_pickle=True)

data = np.load('/home/cat/IA1pm_Feb10_30Hz_rnn_training.npz',
             allow_pickle=True)

# np.save('/home/cat/test_data.npy',d)

# prep data

d = np.float64(data['data_segments'][:,:930].transpose(0,2,1))[:,roi_id]
#d = data
print ("data shape (# of trials, # of time points): ", d.shape)
print ("any nans: ", np.isnan(d).sum())

# prep plots
fig=plt.figure()
ax=plt.subplot(121)

# plot mean of time series
m = np.mean(d,axis=0)
plt.plot(m, c='black')
plt.title("mean is resulting in 1E-12, i.e. zeros")

# remove a single random time series from the data and replot mean
ax=plt.subplot(122)
idx = np.random.randint(0,22,1)
m = np.mean(np.delete(d,idx,axis=0),axis=0)
plt.plot(m, c='black')
plt.title("mean is correct given values")

# 
plt.show()



data shape (# of trials, # of time points):  (13, 930)
any nans:  0


In [4]:
# ##########################################################
# ######## PLOT SINGLE SESSION SVM DECODING SESSION ########
# ##########################################################
# vis.significance = 0.05
# vis.linewidth=10

# #
# vis.cbar_offset = 0
# vis.smooth_window = 30
# vis.lockout = False
# vis.xvalidation = 10
# vis.sliding_window = 30

# # 
# vis.min_trials = 10

# ############################
# # Fig 2E
# vis.window = 30

# # examples > 10 sec
# #vis.animal_id = "AQ2"
# #vis.session_id = 'Feb12_'
# #vis.session_id = 'Mar15_'
# #vis.session_id = 'Mar18_'
# #vis.session_id = 'Apr4_'
# #vis.session_id = 'Apr22_'


# vis.cbar_thick = 0.05

# # # 


# # # 
# # vis.save_fname = '/home/cat/predict.png'

# alpha = 1.0
# #sessions = ['IJ2pm_Feb29_30Hz','IJ2pm_Mar1_30Hz','IJ2pm_Mar2_30Hz','IJ2pm_Mar3_30Hz','IJ2am_Mar7_30Hz' ]
# session = 'IJ2pm_Mar3_30Hz'
# vis.animal_id = 'IJ2'
# vis.fname = None

# vis.code = 'code_04'
# vis.pickle = False
# plot_single = False

# vis.fig=plt.figure(figsize=(10,10))
# vis.ax = plt.subplot(111)

# if plot_single:
#     print ("SESSION: ", session)
#     fname_out = 'single'
#     vis.n_trials_plotting=[]
#     vis.shift_SVM = True
#     vis.n_trials=200
#     vis.session_id = session
#     vis.shift_flag = False
#     vis.apply_lever_offset = False   # this is for sihfting DLC triggered data
#     vis.xlim = [-15,0]
#     vis.show_EDT = True
#     vis.edt_offset_y = .1
#     vis.edt_offset_x = 1.5#
#     vis.show_title = False
#     vis.show_legend = False
#     vis.legend_location = 1
#     vis.cbar=False

#     #vis.get_sessions()
#     #print (vis.sessions)
#     # 
#     vis.cmap = "Blues_r"
#     vis.alpha = alpha
#     vis.plot_significant('blue',vis.session_id)

# else:
#     fname_out = 'concatenated'

# #     print ("")
# #     vis.session_id = 'concatenated'
# # #     vis.fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenatedcode_04_trial_ROItimeCourses_30sec_Xvalid5_Slidewindow30_5PCA_5PCA.npz'
# # #     vis.cmap = "Greens_r"
# #     vis.title = '5pca'
# #     vis.title_offset = 0
# #     vis.cbar=True
# # #     vis.plot_significant_concatenated('green',vis.session_id)

#     vis.n_trials=200
#     vis.session=session
#     vis.session='concatenated'
#     vis.session_id = 'concatenated'
#     vis.fname = '/media/cat/4TBSSD/yuki/IJ2/SVM_Scores/SVM_Scores_concatenated_code_04_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz'
#     vis.cmap = "Reds_r"
#     vis.xlim = [-15,0]
#     vis.title = '20pca'
#     vis.edt_offset_y = .1
#     vis.edt_offset_x = 3
#     vis.cbar = False
#     vis.title_offset = -0.05
#     vis.plot_significant_concatenated('red',vis.session_id)

#     vis.cbar_offset=0

# if True:
#     plt.savefig('/home/cat/'+session+'.svg',dpi=1200)
#     plt.close()
# else:
#     plt.show()


In [5]:
# ###############################################################
# ####### GENERATE LONGITUDINAL SVM CURVES (VIRIDIS PLOTS) ######
# ###############################################################

# # 6 panels with SVM curves in vidiris; ALREADY GEENERATED
# names = ['IA1','IA2','IA3','IJ1','IJ2', 'AQ2']

# fig, ax = plt.subplots()
# import scipy
# from statsmodels.stats.multitest import multipletests

# smooth = True
# smooth_window = 30
# significance = 0.05
# auc = []
# early = []
# from tqdm import tqdm, trange
# for k, name in enumerate(names):
#     fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

#     ax=plt.subplot(2,3,k+1)
#     colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

#     auc.append([])
#     early.append([])
#     for ctr,fname in tqdm(enumerate(fnames)):

#         data = np.load(fname, allow_pickle=True)
#         acc = data['accuracy']

#         if smooth:            
#             data = []
#             for p in range(acc.shape[1]):
#                 box = np.ones(smooth_window)/smooth_window
#                 trace_smooth = np.convolve(acc[:,p], 
#                                            box, 
#                                            mode='valid')

#                 data.append(trace_smooth)
#             data = np.array(data)
#             acc = np.array(data).copy().T

#         # 
#         mean = acc.mean(1)

#         t = np.arange(mean.shape[0])/30-29
#         ax.plot(t,mean,
#                 linewidth=2,
#                 color=colors[ctr])
    
#         # append area under cruve up to t = -10sec
#         #print (mean.shape)
#         temp = mean[-300:]  # last 10sec prior to movement
#         auc[k].append(temp.sum())
        
#         sig, earliest = compute_significance(data.T,
#                                              significance)
#         early[k].append(earliest)
        
#     plt.ylim(0.4,1.0)
#     plt.xlim(-15,0)
#     plt.plot([-30,0],[0.5,0.5],'--',c='black')

# if True:
#     plt.savefig('/home/cat/edt_trends.svg',dpi=1200)
#     plt.close()


In [6]:
# #########################################
# ########### EDT FITS ####################
# #########################################
# # EDT FITS

# colors = ['black','blue','red','green','magenta','pink','cyan']
# ids= ["M1","M2","M3","M4","M5","M6",]
# fig=plt.figure(figsize=(6,6))
# for k in range(len(early)):
# #for k in [3]:
#     ax=plt.subplot(3,2,k+1)
#     temp = np.array(early[k])
#     idx =np.where(temp<-20)[0]
#     temp[idx]=temp[idx-1]
#     t = np.arange(temp.shape[0])/temp.shape[0]
    
#     #
#     lr = LinearRegression()
#     lr.fit(t.reshape(-1, 1), temp.reshape(-1, 1))
#     #print (k, temp)
#     #print (k,"tsta: ", lr.t, "pval: ", lr.p)


    
#     temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
#     plt.plot(t, temp2,
#              linewidth=6,
#              #label=ids[k]+ " ***** ",
#              c='black')

#     # 
#     corr = scipy.stats.pearsonr(t,temp)
#     print ("corrL: ", corr)
    
#     plt.scatter(t,
#                 temp, 
#                 #label=names[k]+ " "+str(round(corr[0],2))+
#                 #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
#                 s=100,
#                 #linewidth=4,
#                 c='grey',
#                 alpha=.8,
#                label = str(round(corr[0],2)))
#     #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)

#     plt.xticks([])
#     plt.yticks([])
#     legend = plt.legend(handlelength=0, fontsize=12)
    
#     #plt.legend(fontsize=12)
#     plt.ylim(-12,0)

# #
#     plt.xlim(0,1)

# if True:
#     plt.savefig('/home/cat/edt_fits.svg', dpi=600)
#     plt.close()
# else:
#     plt.show()


In [26]:
#############################################################
### COMPUTE DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ##
#############################################################
from tqdm import trange

# GENERATE DATA FOR ACCORDEON PLOTS or EDT CURVES

vis = Visualize.Visualize()
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 30
vis.lockout_window = 10
vis.code = 'code_04'
vis.lockout = False
vis.imaging_rate = 30.
vis.pca_var = 0.95
vis.pca_flag = True
vis.xvalidation = 10
vis.sliding_window = 30

# 
vis.smooth_window = 30
vis.min_trials = 20
vis.xlim=[-15,0]
vis.pickle = False
vis.shift_flag = False
vis.apply_lever_offset = False
vis.dlc_offset_flag = False
vis.concatenated_flag = False
vis.compute_roi = True

# 
vis.ideal_window_flag = False
vis.regular_flag = True

#
vis.significance=0.01

vis.overwrite=True
# 
codes = ['_Retrosplenial', '_barrel', '_limb', '_visual','_motor']
#codes = ['limb, layer 1 - right', 'limb, layer 1 - left']

       
# compute regular sessions
for animal_id in vis.animal_ids:
    vis.animal_id = animal_id
    
    for code in codes:
        vis.code = code
        vis.compute_first_decoding_time_ROI()



    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb1_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb4_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb5_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb12_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb19_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb22_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xval

 ... finished session...edt:  -3.1333333333333333

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb18_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  14
Insufficient trials... 14
self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz/IA1pm_Feb19_30Hz__Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz/IA1pm_Feb19_30Hz__Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz__Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz__Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBS

 ... finished session...edt:  0.0

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Mar30_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  17
Insufficient trials... 17
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Mar31_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  14
Insufficient trials... 14
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Apr1_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  13
Insufficient trials... 13
self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Apr4_30Hz/IA1pm_Apr4_30Hz__Retrosplen

 ... finished session...edt:  0.0

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1am_May18_Week6_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  16
Insufficient trials... 16
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1am_May20_Week6_30Hz_Retrosplenial_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  10
Insufficient trials... 10
 # OF GOOD SESSIONS:  30





    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb1_30Hz_barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm_scores results:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb4_30Hz_barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
    missing svm

 ... finished session...edt:  -1.0333333333333334

self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_ROI_IA1pm_Feb18_30Hz_barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  14
Insufficient trials... 14
self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz/IA1pm_Feb19_30Hz__barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz/IA1pm_Feb19_30Hz__barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz__barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz__barrel_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/IA1/SVM_Scores/SVM_Scores_R

...

 ... finished session...edt:  -2.433333333333333

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr26_Week8_30Hz_visual_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  112
(1742,)
self mean:  (450,)
 ... finished session...edt:  0.0

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr27_Week8_30Hz_visual_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  138
(1742,)
self mean:  (450,)
 ... finished session...edt:  0.0

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr28_Week8_30Hz_visual_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  86
(1742,)
self mean:  (450,)
 ... 

 ... finished session...edt:  -1.3666666666666667

self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ

 ... finished session...edt:  -2.7

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2pm_Jan28_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  61
(1742,)
self mean:  (450,)
 ... finished session...edt:  -1.2333333333333334

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Jan29_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  41
(1742,)
self mean:  (450,)
 ... finished session...edt:  -2.566666666666667

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2pm_Jan29_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  51
(1742,)
self mean:  (450,)
 ... finish

 ... finished session...edt:  -2.3

self.fname:  /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Mar16_Week2_30Hz/AQ2am_Mar16_Week2_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
 ... data missing npy /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Mar16_Week2_30Hz/AQ2am_Mar16_Week2_30Hz__motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Mar17_Week2_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  105
(1742,)
self mean:  (450,)
 ... finished session...edt:  -1.8666666666666667

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Mar18_Week2_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  141
(1742,)
self mean:  (450,)
 ... fin

 ... finished session...edt:  -2.033333333333333

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr26_Week8_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  112
(1742,)
self mean:  (450,)
 ... finished session...edt:  -1.9

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr27_Week8_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  138
(1742,)
self mean:  (450,)
 ... finished session...edt:  -2.2333333333333334

self.fname:  /media/cat/4TBSSD/yuki/AQ2/SVM_Scores/SVM_Scores_ROI_AQ2am_Apr28_Week8_30Hz_motor_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz
DATA;  (1771, 10)
... DLC shift applied for lever pull:  0
 post trials data size  (1771, 10)
      n-trials:  86
(1742,)
self mean:

In [10]:
#############################################################
#### PLOT DISTRIBUTION OF FIRST SIG DECODING ALL ANIMALS ####
#############################################################

# REPLACEMENT OF ACORDEON PLOTS
# plot_edt_distributions()     _ older histogram curve plots

def plot_edt_distributions_box_plots():

    codes = ['_Retrosplenial', '_barrel', '_limb', '_visual','_motor','']
    #codes = ['limb, layer 1 - right', 'limb, layer 1 - left']
    clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']


    vis = Visualize.Visualize()
    
    fig = plt.figure(figsize=(20,4))
    legend_size = 22
    for ctr, animal_id in enumerate(vis.animal_ids):
        ax=plt.subplot(1,6,ctr+1)
        fnames = []
        
        for code in codes:
            fnames.append('/media/cat/4TBSSD/yuki/'+animal_id+'/first_decoding_time'+code+'.npz')

        # 
        shifts=[+15,+15,+15,+15,+15,+15,+15]
        linestyles = ['-','-','-','-','-','-','-','-']
        
        #titles = ['all','lockout','concatenated']
        pvals = [0.05,0.01,0.001,0.0001,0.00001]
        edts = []
        yvals = []

        # 
        for ctr_type, fname in enumerate(fnames):
            # 
            print ("Fname; ", fname)
            fname = fname.replace("time_","time__")
            res = np.load(fname, allow_pickle=True)['all_res_continuous']

            # 
            #clr = vis.colors[ctr_type]
            #plotting = False
            #res = vis.plot_first_decoding_time_curves(data,
            #                                            codes[ctr_type],
            #                                            shifts[ctr_type],
            #                                            linestyles[ctr_type],
            #                                            clr, 
            #                                            plotting)
            
            print ("res: ", res)
            temp = res 
            
            # remove low values; don't use fo rnow
            if False:
                idx = np.where(temp<-0.5)[0]
                temp=temp[idx]

            #    
            edts.append(temp)
            yvals.append(res[1])

        edts_saved = np.array(edts).copy()
        
        if len(codes)!=2:
            my_dict = dict(ret = edts[0], 
                          barrel = edts[1],
                          limb = edts[2],
                          visual = edts[3],
                          motor = edts[4],                      
                          All = edts[5]+15,                      
                         )
        else:
            my_dict = dict(right = edts[0], 
                           left = edts[1])

        
        data = pd.DataFrame.from_dict(my_dict, orient='index')
        data = data.transpose()

                
        
        ######################################################### 
        ################## SCATTER PLOTS ########################
        ######################################################### 
        for i,d in enumerate(data):
            y = data[d]
            x = np.random.normal(i+1, 0.04, len(y))
            plt.plot(x, y, 
                     mfc =clrs_local[i], 
                     mec='k', 
                     ms=7, 
                     marker="o", 
                     linestyle="None",
                     alpha=.8
                     
                    )

            
        if animal_id=='IA1':
            patches = []
            for c,code in enumerate(codes):
                patches.append(mpatches.Patch(color=clrs_local[c], label=code))
        
        
            plt.legend(handles=patches,fontsize=legend_size)

            
        ######################################################### 
        ###################### BOX PLOTS ########################
        ######################################################### 
        flierprops = dict(marker='o', 
                          #markerfacecolor='g', 
                          #markersize=10000,
                          linestyle='none', 
                          markeredgecolor='r')
        
        #
        data.boxplot(showfliers=False,
                    flierprops=flierprops)

        temp = np.hstack(yvals)
        ymax = np.max(temp)

        #
        plt.xlim(0.5, 6.5)
        plt.ylim(-15,0)    
        plt.xticks([])
        plt.yticks([])

        #
        plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
        plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)

        # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

        patches = []
        if len(codes)!=2:
            for p in [0,2,4]:
                res = stats.ks_2samp(edts_saved[p], edts_saved[3])
                print ("res: ", res)
                label_ = ''
                for k in range(len(pvals)):
                    if res[1]<pvals[k]:
                        label_ = label_ + "*"
                    else:
                        break

                patches.append(mpatches.Patch(label=label_))
                
            res = stats.ks_2samp(edts_saved[2], edts_saved[5])
            print ("res: ", res)
            print ("")
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='blue', label=label_))
            
            
        else:
            res = stats.ks_2samp(edts_saved[0], edts_saved[1])
            print ("ks test: ", res)
            label_ = ''
            for k in range(len(pvals)):
                if res[1]<pvals[k]:
                    label_ = label_ + "*"
                else:
                    break

            patches.append(mpatches.Patch(color='blue', label=label_))


#         ##########################################
#         print ("#####################")
#         print ("edts: ", edts_saved[0], edts_saved[2])
#         res = stats.ks_2samp(edts_saved[0], edts_saved[2])
#         label_ = ''
#         for k in range(len(pvals)):
#             if res[1]<pvals[k]:
#                 label_ = label_ + "*"
#             else:
#                 break

#         patches.append(mpatches.Patch(color='red', label=label_))

        #print ('')
        #ax2 = ax.twinx()  
        #ax.legend(handles=patches,
        #           fontsize=legend_size)
        plt.yticks([])


    ############################################################
    ############################################################
    ############################################################
    if False:
        plt.show()
    else:
        plt.savefig('/home/cat/first_scatter_ROIs.svg')
        plt.close()
        
# 
plot_edt_distributions_box_plots()


Fname;  /media/cat/4TBSSD/yuki/IA1/first_decoding_time_Retrosplenial.npz
res:  [-3.33333333 -3.06666667 -4.43333333 -2.36666667 -2.43333333 -3.13333333
 -3.4        -3.06666667  0.         -0.4        -1.43333333 -1.63333333
  0.         -3.4         0.         -1.         -0.7         0.
  0.          0.          0.          0.          0.          0.
  0.          0.         -1.36666667  0.          0.          0.        ]
Fname;  /media/cat/4TBSSD/yuki/IA1/first_decoding_time_barrel.npz
res:  [-2.66666667 -4.66666667 -2.06666667 -1.36666667 -1.8        -1.03333333
 -2.43333333 -1.1         0.         -1.3         0.         -0.76666667
 -2.86666667 -1.83333333  0.          0.         -1.26666667 -0.3
 -1.73333333 -1.43333333 -1.7        -1.93333333 -2.36666667 -1.2
  0.          0.         -1.43333333  0.         -1.73333333 -1.9       ]
Fname;  /media/cat/4TBSSD/yuki/IA1/first_decoding_time_limb.npz
res:  [-4.06666667 -4.7        -4.06666667 -2.13333333 -4.36666667 -3.86666667
 -2.

<ipython-input-10-b74443254e34>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edts_saved = np.array(edts).copy()
<ipython-input-10-b74443254e34>:107: UserWarning: The handle <matplotlib.patches.Patch object at 0x7f6869946970> has a label of '_Retrosplenial' which cannot be automatically added to the legend.
  plt.legend(handles=patches,fontsize=legend_size)
<ipython-input-10-b74443254e34>:107: UserWarning: The handle <matplotlib.patches.Patch object at 0x7f6869961520> has a label of '_barrel' which cannot be automatically added to the legend.
  plt.legend(handles=patches,fontsize=legend_size)
<ipython-input-10-b74443254e34>:107: UserWarning: The handle <matplotlib.patches.Patch object at 0x7f6869961610> has a label of '_limb' which cannot be automatically added to

res:  KstestResult(statistic=0.14285714285714285, pvalue=0.9440858097815165)
res:  KstestResult(statistic=0.5714285714285714, pvalue=0.00014600755513485803)
res:  KstestResult(statistic=0.35714285714285715, pvalue=0.05551971712876329)
res:  KstestResult(statistic=1.0, pvalue=5.551115123125783e-16)

Fname;  /media/cat/4TBSSD/yuki/IA3/first_decoding_time_Retrosplenial.npz
res:  [-1.76666667 -3.76666667 -3.13333333 -3.43333333 -4.3        -3.63333333
 -3.06666667 -1.86666667 -1.46666667 -1.23333333 -1.63333333 -1.03333333
 -2.66666667  0.         -1.56666667 -1.2        -3.4         0.
 -2.53333333 -1.8        -1.66666667  0.         -1.73333333 -2.8
 -1.56666667 -2.06666667 -3.2         0.         -2.53333333 -3.4
 -1.73333333 -1.63333333 -3.8        -1.43333333 -1.73333333 -4.5
 -1.5        -3.63333333]
Fname;  /media/cat/4TBSSD/yuki/IA3/first_decoding_time_barrel.npz
res:  [-1.06666667 -1.66666667 -1.93333333 -1.5        -2.9        -1.9
 -1.73333333 -1.6        -1.53333333 -1.46666667

Fname;  /media/cat/4TBSSD/yuki/AQ2/first_decoding_time_Retrosplenial.npz
res:  [-1.56666667 -0.96666667 -1.46666667 -1.7        -0.8        -1.8
  0.         -1.86666667  0.          0.          0.         -1.93333333
 -1.93333333  0.         -1.86666667 -1.6        -1.63333333  0.
 -1.6        -1.53333333  0.         -0.63333333  0.          0.
 -3.16666667 -2.16666667 -2.86666667 -4.46666667 -4.2         0.
 -2.23333333  0.          0.         -1.6        -1.63333333 -2.66666667
 -1.9        -2.26666667 -2.13333333  0.          0.          0.
  0.          0.          0.         -3.26666667 -2.1        -2.7
 -0.76666667 -1.7         0.         -3.03333333 -2.46666667 -3.06666667
 -0.96666667 -4.86666667 -2.46666667 -1.23333333 -4.83333333 -1.8
 -1.36666667 -1.93333333 -2.46666667 -2.26666667 -2.8         0.
 -1.86666667 -1.86666667  0.        ]
Fname;  /media/cat/4TBSSD/yuki/AQ2/first_decoding_time_barrel.npz
res:  [ 0.          0.         -0.8        -1.73333333  0.         -1.76666

<ipython-input-10-b74443254e34>:64: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  edts_saved = np.array(edts).copy()


In [126]:
############################################################
#### PLOT CORRELATION OF DECODING TIME AND NO. SESSIONS ####
############################################################
from tqdm import trange
import matplotlib.cm as cm

vis.smooth_window = 30
vis.min_trials = 20
vis.animal_id = 'IA1'
vis.session = "Feb1_"
vis.window = 30
vis.xlim=-15

#
fig = plt.figure(figsize=(10,8))
vis.significance=0.05 #1E-2
vis.lockout=False
vis.ylims=[200,400]
vis.cmap=cm.Reds
vis.plot_legend = False
# 
clr='blue'

# TWO OPTIONS: singles or contatnated
#fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_all_30sec_backup.npz'
fname = '/media/cat/4TBSSD/yuki/first_decoding_time_minTrials20_lockout_30sec_concatenated_backup.npz'
vis.plot_first_decoding_time_vs_n_trials(clr, fname)

# #
if False:
    plt.show()
else:
    plt.savefig('/home/cat/correaltions_concatenated.svg',dpi=1200)
    plt.close()


IA1 Feb1_ [ -4.2         -6.63333333  -4.36666667  -4.63333333  -4.86666667
 -11.43333333 -10.13333333  -6.66666667  -4.43333333  -5.06666667
  -4.2         -4.16666667  -4.33333333  -4.4         -6.33333333
  -4.7         -3.86666667  -4.3         -3.56666667  -3.9
  -3.36666667  -3.96666667  -3.56666667  -3.73333333  -3.86666667
  -3.63333333  -4.26666667  -3.3         -2.86666667  -3.96666667
  -3.66666667  -4.13333333  -3.36666667  -3.26666667  -2.83333333
  -3.63333333  -3.03333333  -3.          -2.7         -3.
  -2.9         -2.66666667  -3.6         -2.43333333  -3.
  -2.46666667  -2.53333333  -2.5         -2.5         -2.16666667
  -2.          -2.          -2.        ]
all preds:  (53, 1)
IA1 Feb1_ [-4.53333333 -4.33333333 -3.13333333 -2.63333333 -2.66666667 -2.
 -2.8        -2.4        -3.13333333 -3.66666667 -2.         -2.
 -2.         -4.96666667 -4.8        -4.83333333 -4.76666667 -5.2
 -8.63333333 -7.36666667 -7.4        -7.2        -7.8        -6.9
 -6.         -4.7   

In [87]:
###################################################
############# PLOT PREDICTOIN TRENDS ##############
###################################################

# FIG 3I  3J
names = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

auc = []
fig, ax = plt.subplots()

for k, name in enumerate(names):
    fnames = np.loadtxt('/media/cat/4TBSSD/yuki/'+name+'/concatenated_svm.txt',dtype='str') #[::-1]

    ax=plt.subplot(2,3,k+1)
    colors = plt.cm.viridis(np.linspace(0,1,len(fnames)))

    smooth = True
    smooth_window = 30
    auc.append([])
    for ctr,fname in enumerate(fnames):

        data = np.load(fname, allow_pickle=True)
        acc = data['accuracy']
        #print (acc.shape)

        if smooth:            
            data = []
            for p in range(acc.shape[1]):
                box = np.ones(smooth_window)/smooth_window
                trace_smooth = np.convolve(acc[:,p], 
                                           box, 
                                           mode='valid')

                data.append(trace_smooth)

            acc = np.array(data).copy().T

        # 
        mean = acc.mean(1)

        t = np.arange(mean.shape[0])/30-29
        ax.plot(t,mean,
                 linewidth=2,
                color=colors[ctr])
    
        # append area under cruve up to t = -10sec
        print (mean.shape)
        temp = mean[-300:]  # last 10sec prior to movement
        auc[k].append(temp.sum())

# 
fig2=plt.figure()
for k in range(len(auc)):
    plt.plot(auc[k], label=names[k],
            linewidth=4)
plt.legend(fontsize=20)
plt.show()

(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)
(841,)

In [88]:
#########################################
########### AUC AND EARLY PLOTS #########
#########################################

# FIg 3J 3I

colors = ['black','blue','red','green','magenta','pink','cyan']
ids= ["M1","M2","M3","M4","M5","M6",]
fig2=plt.figure()
for k in range(len(early)):
    temp = np.array(early[k])
    idx =np.where(temp<-20)[0]
    temp[idx]=temp[idx-1]
    t = np.arange(temp.shape[0])
    
    #
    temp2 = np.poly1d(np.polyfit(t, temp, 1))(t)
    plt.plot(t, temp2,
             linewidth=6,
             label=ids[k]+ " ***** ",

             c=colors[k])

    # 
    corr = scipy.stats.pearsonr(t,temp2)
    print ("corr: ", corr)
    
    
    plt.scatter(t,
                temp, 
                #label=names[k]+ " "+str(round(corr[0],2))+
                #                        " ("+str("pval: {0:.1}".format(corr[1]))+")",
                s=100,
                linewidth=4,
                c=colors[k],
                alpha=.3)
    #t = np.arange(auc1.shape[0])/(auc1.shape[0]-1)


#
plt.xlim(0,80)
plt.ylim(-12,0)
plt.legend(fontsize=20)
plt.show()


corr:  (1.0, 0.0)
corr:  (-0.9999999999999999, 3.3310483861078736e-244)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-1.0, 0.0)
corr:  (-0.9999999999999999, 0.0)


In [169]:
data = np.load('/media/cat/4TBSSD/yuki/IJ2/tif_files/IJ2pm_Mar2_30Hz/IJ2pm_Mar2_30Hz_whole_stack_trial_ROItimeCourses_15sec_pca30components.npy')
print (data.shape)

(40000, 30)


In [69]:
##############################################
##############################################
##############################################

from scipy.signal import savgol_filter

codes = ['Retrosplenial', 'barrel', 'motor', 'limb', 'visual']
clrs=['black','blue','red','magenta','brown']
smoothing = True
min_trials = 30
    
######################    
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'AQ2'

sessions = Visualize.get_sessions(root_dir,animal_id,'all')

print ("SESSIONS: ", sessions)
ctr_sess=0
fig = plt.figure()
for session_id in sessions:
    fnames = []
    for k in range(len(codes)):
        fnames.append(root_dir+ animal_id+'/SVM_Scores/SVM_Scores_ROI_'+
                      session_id+"_"+
                      codes[k]+'_trial_ROItimeCourses_30sec_Xvalid10_Slidewindow30.npz')

    try:
        temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,
                              session_id+"_all_locs_selected.txt"), dtype='str')
    except:
        print (" ntrial file missing: ")
        continue
        
    if len(temp)<min_trials:
        print ("too few trials: ", len(temp))
        continue

    ax=plt.subplot(8,10,ctr_sess+1)
    for ctr,fname in enumerate(fnames):
        try:
            data = np.load(fname, allow_pickle=True)
        except:
            print ("SVM not compued yet, skipping")
            continue

        acc_r = data['accuracy']

        if smoothing:
            for k in range(acc_r.shape[1]):
                acc_r[:,k] = savgol_filter(acc_r[:,k], 7, 3)

        #
        mean_r = acc_r[:900].mean(1)
        std_r = np.std(acc_r[:900],axis=1)

        t = np.arange(mean_r.shape[0])/30-29

        plt.plot(t,mean_r, linewidth=3,
                label=codes[ctr])
        if False:
            ax.fill_between(t, mean_r+std_r,
                           mean_r-std_r,
                           color=clrs[ctr],
                           alpha=.2)
    # 
    if ctr_sess==0:
        plt.legend()
    else:
        plt.xticks([])
        plt.yticks([])

    plt.ylabel("# trials: "+str(len(temp)))
    plt.xlim(-5,0)
    plt.ylim(0,1.0)
    plt.plot([-30,0],[0.5,0.5],'--',c='grey')

    ctr_sess+=1
   #except:
   #     pass
    
plt.show()



SESSIONS:  ['AQ2am_Dec9_30Hz' 'AQ2am_Dec10_30Hz' 'AQ2pm_Dec10_30Hz'
 'AQ2am_Dec11_30Hz' 'AQ2pm_Dec14_30Hz' 'AQ2am_Dec14_30Hz'
 'AQ2pm_Dec16_30Hz' 'AQ2am_Dec17_30Hz' 'AQ2pm_Dec17_30Hz'
 'AQ2am_Dec18_30Hz' 'AQ2pm_Dec18_30Hz' 'AQ2am_Dec21_30Hz'
 'AQ2am_Dec22_30Hz' 'AQ2am_Dec23_30Hz' 'AQ2am_Dec28_30Hz'
 'AQ2am_Dec29_30Hz' 'AQ2am_Dec30_30Hz' 'AQ2am_Dec31_30Hz'
 'AQ2am_Jan4_30Hz' 'AQ2am_Jan5_30Hz' 'AQ2am_Jan6_30Hz' 'AQ2am_Jan7_30Hz'
 'AQ2am_Jan8_30Hz' 'AQ2pm_Jan11_30Hz' 'AQ2am_Jan11_30Hz'
 'AQ2pm_Jan12_30Hz' 'AQ2am_Jan12_30Hz' 'AQ2pm_Jan13_30Hz'
 'AQ2am_Jan13_30Hz' 'AQ2pm_Jan14_30Hz' 'AQ2am_Jan14_30Hz'
 'AQ2am_Jan15_30Hz' 'AQ2pm_Jan15_30Hz' 'AQ2pm_Jan18_30Hz'
 'AQ2am_Jan18_30Hz' 'AQ2am_Jan19_30Hz' 'AQ2pm_Jan19_30Hz'
 'AQ2pm_Jan20_30Hz' 'AQ2am_Jan20_30Hz' 'AQ2pm_Jan21_30Hz'
 'AQ2am_Jan21_30Hz' 'AQ2pm_Jan22_30Hz' 'AQ2am_Jan22_30Hz'
 'AQ2am_Jan25_30Hz' 'AQ2pm_Jan25_30Hz' 'AQ2pm_Jan26_30Hz'
 'AQ2am_Jan26_30Hz' 'AQ2am_Jan27_30Hz' 'AQ2pm_Jan27_30Hz'
 'AQ2am_Jan28_30Hz' 'AQ2pm_Jan28_30Hz' 'AQ2am_Ja

<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec30_30Hz/AQ2am_Dec30_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


too few trials:  4
too few trials:  27
too few trials:  18
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
too few trials:  26
 ntrial file missing: 
too few trials:  13
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  2
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  4
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
 ntrial file missing: 
too few trials:  3
too few trials:  21
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
too few trials:  0


<ipython-input-69-81a02e332188>:29: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb25_30Hz/AQ2am_Feb25_30Hz_all_locs_selected.txt"
  temp = np.loadtxt(os.path.join(root_dir,animal_id,'tif_files',session_id,


SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping
SVM not compued yet, skipping


In [85]:
data = np.load('/media/cat/4TBSSD/yuki/AQ2/alldata_body_and_nonreward_lockout_21secLockout_[0, 1, 5]bodyfeats.npz',
              allow_pickle=True)
names = data['names']
print (len(names))

# 
trials = np.vstack(data['trials'])
print (trials.shape)



39
(1366, 1800, 16)


In [ ]:
data = np.load('/media/cat/4TBSSD/yuki/IA1/alldata_body_and_nonreward_lockout_0secLockout_[]bodyfeats_0_super_res.npz')